get quotations from chronicling america

In [217]:
from pprint import pprint as pp
from tqdm import tqdm
import json
import time
import random
import pathlib
import re
import json
from pprint import pprint as pp
import numpy as np


In [269]:
import spacy
from spacy import displacy
from nltk.tokenize import word_tokenize
from collections import Counter, defaultdict

nlp = spacy.load("en_core_web_lg")


Exception ignored in: <function tqdm.__del__ at 0x1088bce50>
Traceback (most recent call last):
  File "/Users/ryanbrate/miniconda3/envs/mexico/lib/python3.10/site-packages/tqdm/std.py", line 1149, in __del__
    self.close()
  File "/Users/ryanbrate/miniconda3/envs/mexico/lib/python3.10/site-packages/tqdm/std.py", line 1278, in close
    if self.last_print_t < self.start_t + self.delay:
AttributeError: 'tqdm' object has no attribute 'last_print_t'


In [171]:
def search_exact_phrase_single_page(page_number, phrase = "negro+said"):

    url = f"https://chroniclingamerica.loc.gov/search/pages/results/?rows=100&searchType=basic&phrasetext={phrase}&page={page_number}&sort=relevance&format=json"
    try:
        response = requests.get(url,timeout=30)  # Including a timeout for the request
        data = response.json()
        if data['items']:
            return data['items']
        else:
            raise
            
    except requests.Timeout:
        return False


# Extract the quotations in context

## phrasetext=negro+said

https://chroniclingamerica.loc.gov/search/pages/results/?state=&date1=1756&date2=1963&phrasetext=negro+said&x=0&y=0&dateFilterType=yearRange&rows=20&searchType=basic

... we see 10120 results

In [170]:
# Note 1: experience shows that 100 rows at a time, with a 30 sec timeout and 1 minute in between works well.
# Note 2: 10120 results / 100 = 102 pages
for page_number in tqdm(range(1, 102+1)):
    search_results = False
    while search_results == False:
        time.sleep(30)
        search_results = search_exact_phrase_single_page(page_number, phrase="negro+said")
    with open(f"chunk_{page_number}.json", "w") as f:
        json.dump(search_results, f)
    time.sleep(60)


100%|█████████████████████████████████████████████| 5/5 [06:28<00:00, 77.75s/it]


## phrasetext=said+the+negro

https://chroniclingamerica.loc.gov/search/pages/results/?state=&date1=1756&date2=1963&phrasetext=said+the+negro&x=0&y=0&dateFilterType=yearRange&rows=100&searchType=basic

... we see 14540 results

In [297]:
# Note 1: experience shows that 100 rows at a time, with a 30 sec timeout and 1 minute in between works well.
# Note 2: 14540 results / 100 = 146 pages
# for page_number in tqdm(range(108, 146+1)):
for page_number in [65, 107]:
    search_results = False
    while search_results == False:
        time.sleep(30)
        search_results = search_exact_phrase_single_page(page_number, phrase="said+the+negro")
    with open(f"chunk_{page_number}.json", "w") as f:
        json.dump(search_results, f)
    time.sleep(60)


## join them all together

In [298]:
corpus = []
for dir in [pathlib.Path("phrase_text_negro+said/"), pathlib.Path("phrase_text_said+the+negro/")]:

    fps = list(dir.glob("*.json"))
    
    for fp in tqdm(fps, desc=str(dir)):
        with open(fp, 'r') as f:
            corpus += json.load(f)

phrase_text_said+the+negro: 100%|████████████| 146/146 [00:01<00:00, 108.81it/s]


In [311]:
len(corpus)

23140

## Filter out repeated pages

In [317]:
corpus_ = []
seen_ids = set([])
for item in tqdm(corpus):
    try:
        id_ = item['id']
        if id_ in seen_ids:
            pass
        else:
            corpus_.append(item)
            seen_ids.add(id_)
    except:
        pass
corpus=corpus_

100%|████████████████████████████████| 23140/23140 [00:00<00:00, 1037058.11it/s]


In [318]:
len(corpus)

22824

### Examine

In [312]:
print(corpus[1])
# 'id' is the unique identifier 

{'sequence': 1, 'county': ['Montgomery'], 'edition': None, 'frequency': 'Daily (except Sun.)', 'id': '/lccn/sn88077225/1951-05-07/ed-1/seq-1/', 'subject': ['African Americans--Ohio--Dayton--Newspapers.', 'African Americans.--fast--(OCoLC)fst00799558', 'Dayton (Ohio)--Newspapers.', 'Montgomery County (Ohio)--Newspapers.', 'Ohio--Dayton.--fast--(OCoLC)fst01204702', 'Ohio--Montgomery County.--fast--(OCoLC)fst01211918'], 'city': ['Dayton'], 'date': '19510507', 'title': 'The daily express.', 'end_year': 1955, 'note': ['Archived issues are available in digital format from the Library of Congress Chronicling America online collection.'], 'state': ['Ohio'], 'section_label': '', 'type': 'page', 'place_of_publication': 'Dayton, Ohio', 'start_year': 1950, 'edition_label': '', 'publisher': 'Paige H. Strickland', 'language': ['English'], 'alt_title': [], 'lccn': 'sn88077225', 'country': 'Ohio', 'ocr_eng': 'it/-\nf.\nVf^ *\'t\n-V ,/•\nI c*.\'\\\nVV,t\n-U^ -V V 7~*.\nA 4\nV .\'•.\nn\nf4 U\n$ e &\nzm\

In [319]:
# how many entries do we have
ids = set()
for item in tqdm(corpus):
    try:
        id_ = item['id']
        text = item['ocr_eng'].replace("\n", "")
        if "negro said" in text or "said the negro" in text:
            ids.add(id_)
    except:
        pass
len(ids)

100%|██████████████████████████████████| 22824/22824 [00:01<00:00, 13708.97it/s]


8529

# Capturing (presumably relevant) quotes either side of manner+speaker combinations. 

In [371]:
"hello".find("he", 0)

0

In [375]:
def get_window(text:str, target:str, target_i: int)->str:
        
    # we stop at ...
    #  i) the first period, where we aren't in a quotation
    i = target_i
    
    in_quote = False  # we assume we aren't in a quote currently
    while i < len(text)-1:

        i += 1  # we start hunting for the end, from after the target word
        c = text[i]

        # period found out of quote
        if in_quote == False and (c == '.'):
            break

        # registering that we enter a quote
        if in_quote == False and (c == '"' or c=='“'):
            in_quote = True
        # registering that we are coming out of a quote
        elif in_quote == True and (c == '"' or c == '”'):
            in_quote = False
 
    end_i = i  # where we found our period ... or just ran out of text
        

    # we start at ...
    #  i) the first preceeding period, where we aren't in a quotation
    i = target_i
    in_quote = False
    while i > 0:

        i -= 1
        c = text[i]

        # period found out of quote
        if in_quote == False and (c=='.'):
            break

        # registering that we enter a quote
        if in_quote == False and (c == '"' or c == '“'):
            in_quote = True
        # registering that we are coming out of a quote
        elif in_quote == True and (c == '"' or c == '”'):
            in_quote = False

    if i == 0:
        start_i = 0
    else:
        start_i = i+1  # after presumed period
    return text[start_i:end_i+1]

In [377]:
# quick test

# 1. 
text = 'Extraneous sentence. "hello." the negro said, "how are you?". Extraneous sentence.'
target = "negro said"
i = text.find(target, 0)
while i != -1:
    window = get_window(text, phrase, i)
    if (window.count('"') > 0 and window.count('"') % 2 == 0) or (window.count('”') > 0 and window.count('”')%2 == 0 and window.count('“') > 0 and window.count('“')%2):
        display(window)
    i = text.find(target, i+1)

# 2. 
text = 'Extraneous sentence. "hello." said the negro, "how are you?". Extraneous sentence.'
target = "said the negro"
i = text.find(target, 0)
while i != -1:
    window = get_window(text, phrase, i)
    if (window.count('"') > 0 and window.count('"') % 2 == 0) or (window.count('”') > 0 and window.count('”')%2 == 0 and window.count('“') > 0 and window.count('“')%2):
        display(window)
    i = text.find(target, i+1)

' "hello." the negro said, "how are you?".'

' "hello." said the negro, "how are you?".'

In [378]:
# let's take a look ...
texts = []
ids = [] # where ids[i] corresponds to texts[i]
for item in tqdm(corpus):
    for phrase in ["negro said", "said the negro"]:
        try:
            
            text = item['ocr_eng'].replace("\n", "")
            id_ = item['id']
    
            # get the window containing the phrase text
            i = text.find(phrase, 0)       
            while i != -1:

                # get next window
                window = get_window(text, phrase, i)
                if (window.count('"') > 0 and window.count('"') % 2 == 0) or (window.count('”') > 0 and window.count('”')%2 == 0 and window.count('“') > 0 and window.count('“')%2):
                    texts.append(window)
                    ids.append(id_)

                # increment i
                i = text.find(phrase, i+1)

    
        except:
            pass

SyntaxError: '(' was never closed (1741219254.py, line 12)

In [334]:
len(texts)

2308

In [335]:
ids

['/lccn/sn83025296/1925-08-13/ed-1/seq-3/',
 '/lccn/sn96075050/1910-06-23/ed-1/seq-8/',
 '/lccn/sn83025182/1910-05-22/ed-1/seq-6/',
 '/lccn/sn84038335/1902-06-12/ed-1/seq-4/',
 '/lccn/sn87062268/1917-07-02/ed-1/seq-4/',
 '/lccn/sn87062055/1914-12-25/ed-1/seq-4/',
 '/lccn/sn99062010/1907-06-13/ed-1/seq-5/',
 '/lccn/sn99062010/1919-04-03/ed-1/seq-3/',
 '/lccn/sn83030214/1900-07-01/ed-1/seq-46/',
 '/lccn/sn86064255/1908-11-20/ed-1/seq-5/',
 '/lccn/sn88084272/1916-03-01/ed-1/seq-20/',
 '/lccn/sn83025020/1909-03-13/ed-1/seq-2/',
 '/lccn/sn83030214/1900-10-07/ed-1/seq-46/',
 '/lccn/sn83040592/1913-06-19/ed-1/seq-7/',
 '/lccn/sn93061371/1913-07-11/ed-1/seq-7/',
 '/lccn/sn87065165/1903-08-07/ed-1/seq-4/',
 '/lccn/sn83045774/1921-03-25/ed-1/seq-9/',
 '/lccn/sn83045774/1921-03-25/ed-1/seq-9/',
 '/lccn/sn84026897/1922-09-13/ed-1/seq-6/',
 '/lccn/sn87076843/1913-07-25/ed-1/seq-3/',
 '/lccn/sn86063758/1912-12-31/ed-1/seq-5/',
 '/lccn/sn86063758/1916-09-19/ed-1/seq-4/',
 '/lccn/sn95063969/1913-07-10

In [336]:
texts

[')(TOPHscattrrially AppraUmJApplying for a divorce, an old Georgia negro said to the judge : "Hit onlya string er fish ter kittied. Jedge ; hut Lawdy. jedge. I\'d givea whale ter git rid er her "—BostonTranscript.O',
 '”All Is Not Lion That Roars.A uegro \\Vas arrested for stealingcoal and employed a lawyer of loudoratorical voice to defeud him In ajustice court.“That lawyer could roar like a Hon,”the negro said. ',
 ' Get rid of allImpurities it the blood by takingHood\'s Sarsaparllla, and thus fortifyyour whole body and prevent IllnessMust Have Made Poof Bargain,Applying for a divorce, an oldGeorgia negro said to the Judge: "Hitonly cost me a string er fish ter gitmarried, jedge, but, please God, I\'dgive a whale ter git rid er her."—Argonaut.B',
 'I alway supposed the mountain roadsto be rough, but they are line no roadin city is better the only\' difllculty isin the width they are just wide enoughwith not a foot to spare, and tf you goover it would be as the old negro said:"where 

# Extracting the quote, manner, speaker

In [400]:
def between(x,t):
    lower, upper = t
    if x >= lower and x <= upper:
        return True
    else:
        return False

def count_quotes(chunk):
    return chunk.count('"') + chunk.count('“') + chunk.count('”')

def yield_type1(text:str, replacement='"oh"', graph = False):

    # chunk into quotes and text-in-between-quotes
    chunks = [chunk for chunk in re.split(r'(".*?")|(“.*?”)', text) if chunk]
    # print('chunks', chunks)

    # get the quotes only
    quotes = [mo.group(0) for mo in re.finditer(r'(".*?")|(“.*?”)', text)]
    # print('quotes', quotes)

    # get counts of quotes in each chunk
    quote_counts = [count_quotes(chunk) for chunk in chunks]
    # print('quote counts in chunks', quote_counts)

    # ensure where a quotations mark occurs, it does so in pairs
    if all([True if (count == 0 or count == 2) else False for count in quote_counts]):

        # build a chunks version with the quotes replaced with replacement
        if replacement == '':
            chunks_ = chunks
        else:
            chunks_ = [chunk if quote_count == 0 else replacement for chunk, quote_count in zip(chunks, quote_counts)]

        # build a text version with quotes replaced
        text_ = "".join(chunks_)
        # print('text_', text_)

        # text as a list of tokens and their properties
        doc = nlp(text_)
        if graph:
            displacy.render(doc, style="dep", jupyter=True)
        
        tokens = {token.i: {'text':token.text, 'modifiers':list([m.i for m in token.children]), 'pos':token.pos_, 'head':token.head.i, 'dep':token.dep_, 'idx':token.idx} for token in doc}
        if graph:
            pp(tokens)
        
        # list of indices which are opening quotation marks
        marks_i = [i for i, d in tokens.items() if d['text']=='"' or d['text']=='“' or d['text']=='”']
        # print('marks', marks_i)
        opening_marks_i = [marks_i[x] for x in range(0,len(marks_i),2)]
        # print('opening quotation marks', opening_marks_i)
        
        # list of indices which are verbs
        verbs_i = [i for i, d in tokens.items() if d['pos']=="VERB"]
        # print('verbs', verbs_i)

        # build a list of (quote, manner, speaker)
        extracts = []
        for r, o in enumerate(opening_marks_i):
            
            o_matched = False
            
            # find a verb which is modified by the opening quotation ...
            for v in verbs_i:
                if o in tokens[v]['modifiers']:
                    # for that verb, find a corresponding speaker, i.e., subj or dobj which modifies the verb
                    for m in tokens[v]['modifiers']:
                        if tokens[m]['dep']=="dobj" or tokens[m]['dep']=="nsubj":
                            extracts.append((quotes[r], tokens[v]['text'], tokens[m]['text']))
                            o_matched = True

            if o_matched == False:
                extracts.append((quotes[r], None, None))

        # yield
        for t in extracts:
            yield t

input_ = '"today seems nice", said Tom'
list(yield_type1(input_))

[('"today seems nice"', 'said', 'Tom')]

In [402]:
# test  1
list(yield_type1('"hello." the negro said, "how are you?".', graph=True))


{0: {'dep': 'punct',
     'head': 5,
     'idx': 0,
     'modifiers': [],
     'pos': 'PUNCT',
     'text': '"'},
 1: {'dep': 'intj',
     'head': 5,
     'idx': 1,
     'modifiers': [],
     'pos': 'INTJ',
     'text': 'oh'},
 2: {'dep': 'punct',
     'head': 5,
     'idx': 3,
     'modifiers': [],
     'pos': 'PUNCT',
     'text': '"'},
 3: {'dep': 'det',
     'head': 4,
     'idx': 5,
     'modifiers': [],
     'pos': 'DET',
     'text': 'the'},
 4: {'dep': 'nsubj',
     'head': 5,
     'idx': 9,
     'modifiers': [3],
     'pos': 'PROPN',
     'text': 'negro'},
 5: {'dep': 'ROOT',
     'head': 5,
     'idx': 15,
     'modifiers': [0, 1, 2, 4, 6, 7, 8, 9, 10],
     'pos': 'VERB',
     'text': 'said'},
 6: {'dep': 'punct',
     'head': 5,
     'idx': 19,
     'modifiers': [],
     'pos': 'PUNCT',
     'text': ','},
 7: {'dep': 'punct',
     'head': 5,
     'idx': 21,
     'modifiers': [],
     'pos': 'PUNCT',
     'text': '"'},
 8: {'dep': 'intj',
     'head': 5,
     'idx': 22,
    

[('"hello."', 'said', 'negro'), ('"how are you?"', 'said', 'negro')]

In [407]:
# test 2
list(yield_type1('"hello," said the old negro - "how are you?"', graph=True))

{0: {'dep': 'punct',
     'head': 3,
     'idx': 0,
     'modifiers': [],
     'pos': 'PUNCT',
     'text': '"'},
 1: {'dep': 'intj',
     'head': 3,
     'idx': 1,
     'modifiers': [],
     'pos': 'INTJ',
     'text': 'oh'},
 2: {'dep': 'punct',
     'head': 3,
     'idx': 3,
     'modifiers': [],
     'pos': 'PUNCT',
     'text': '"'},
 3: {'dep': 'ROOT',
     'head': 3,
     'idx': 5,
     'modifiers': [0, 1, 2, 6, 7, 8, 9, 10],
     'pos': 'VERB',
     'text': 'said'},
 4: {'dep': 'det',
     'head': 6,
     'idx': 10,
     'modifiers': [],
     'pos': 'DET',
     'text': 'the'},
 5: {'dep': 'amod',
     'head': 6,
     'idx': 14,
     'modifiers': [],
     'pos': 'ADJ',
     'text': 'old'},
 6: {'dep': 'nsubj',
     'head': 3,
     'idx': 18,
     'modifiers': [4, 5],
     'pos': 'NOUN',
     'text': 'negro'},
 7: {'dep': 'punct',
     'head': 3,
     'idx': 24,
     'modifiers': [],
     'pos': 'PUNCT',
     'text': '-'},
 8: {'dep': 'punct',
     'head': 3,
     'idx': 26,
    

[('"hello,"', 'said', 'negro'), ('"how are you?"', 'said', 'negro')]

In [347]:
# get (quote, manner, speaker) tuples
extracts = []
for text, id_ in tqdm(zip(texts, ids)):
    for q,m,s in list(yield_type1(text)):
        if s == "negro":
            extracts.append(("https://chroniclingamerica.loc.gov"+id_+f"#words=negro", (q,m,s)))
len(extracts)

2308it [00:04, 463.76it/s]


906

In [348]:
extracts

[('https://chroniclingamerica.loc.gov/lccn/sn83025182/1910-05-22/ed-1/seq-6/#words=negro',
  ('"Hitonly cost me a string er fish ter gitmarried, jedge, but, please God, I\'dgive a whale ter git rid er her."',
   'said',
   'negro')),
 ('https://chroniclingamerica.loc.gov/lccn/sn87062055/1914-12-25/ed-1/seq-4/#words=negro',
  ('"I hopes dis teaches me alesson."', 'said', 'negro')),
 ('https://chroniclingamerica.loc.gov/lccn/sn99062010/1907-06-13/ed-1/seq-5/#words=negro',
  ('"There\'s a dog under them steps, an\'bo\'s shore mad."',
   'said',
   'negro')),
 ('https://chroniclingamerica.loc.gov/lccn/sn88084272/1916-03-01/ed-1/seq-20/#words=negro',
  ('"I came for Mr. Ivans\'s close."', 'said', 'negro')),
 ('https://chroniclingamerica.loc.gov/lccn/sn83045774/1921-03-25/ed-1/seq-9/#words=negro',
  ('"Yes, suh, that\'s what that mail orderhouse sent me,"', 'said', 'negro')),
 ('https://chroniclingamerica.loc.gov/lccn/sn84026897/1922-09-13/ed-1/seq-6/#words=negro',
  ('"I do not deny having 

# max length of quotations?

In [13]:
from tuples_news import quotes as tuples

In [14]:
max([len(q) for url, (q,m,s) in tuples])

603

# save as json

In [15]:
import json

In [16]:
with open("tuples_news.json", "w") as f:
    json.dump(tuples, f)